In [ ]:
#--------------#
###生成实体类别###
#--------------#

In [ ]:
import pickle
from tqdm import tqdm
from prefixspan import PrefixSpan
#import spacy
#import en_core_web_trf
#import spacy_transformers

#nlp = spacy.load('en_core_web_trf')
root_path = '/home/zhangjs/expriment/TKGist/data/'
dataset = 'GDELT/'

from itertools import combinations, chain

def powerset(iterable, num):
    s = list(iterable)
    return list(chain.from_iterable(combinations(s, r) for r in range(1, num)))


# read entity2id file
raw_e2id_dict = {}
raw_id2e_dict = {}
file = open(root_path+dataset+'entity2id.txt', 'r')
for e2id in file.readlines():
    e, id = e2id.strip().split('	')[:2]
    raw_e2id_dict[e] = int(id)
    raw_id2e_dict[int(id)] = e
print(len(raw_e2id_dict))
print(raw_id2e_dict[0])

e_num = len(raw_e2id_dict)

# read relation2id file
raw_r2id_dict = {}
raw_id2r_dict = {}
file = open(root_path+dataset+'relation2id.txt', 'r')
for r2id in file.readlines():
    r, id = r2id.strip().split('	')[:2]
    raw_r2id_dict[r] = int(id)
    raw_id2r_dict[int(id)] = r
print(len(raw_r2id_dict))
print(raw_id2r_dict[0])

r_num = len(raw_r2id_dict)
print(r_num)

In [ ]:
file = open(root_path+dataset+'train.txt', 'r')
triple_2_t = dict()
for fact in file.readlines():
    s, r, o, t = fact.strip().split('	')[:4]
    s = int(s)
    r = int(r)
    o = int(o)
    if (s,r,o) not in triple_2_t.keys():
        triple_2_t[(s,r,o)] = []
    triple_2_t[(s,r,o)].append(t)
    if (o,r,s) not in triple_2_t.keys():
        triple_2_t[(o,r,s)] = []
    triple_2_t[(o,r,s)].append(t)

print(len(triple_2_t))
repeat_num = 0
for key in triple_2_t.keys():
    if len(triple_2_t[key]) > 1:
        repeat_num += 1
print(repeat_num)

In [ ]:
# 获取原始关系序列
import random
# read relation sets (注：存在新实体)
seen_e_set = set()
seen_r_set = set()
raw_e2rset_dict = {}
raw_r_type_set = set()
raw_r_type_list = []
raw_e_2_type_list = []
file = open(root_path+dataset+'train.txt', 'r')
for fact in file.readlines():
    s, r, o, t = fact.strip().split('	')[:4]
    s = int(s)
    r = int(r)
    o = int(o)
    t = int(t)
    
    if s not in raw_e2rset_dict.keys():
        raw_e2rset_dict[s] = set()
    if o not in raw_e2rset_dict.keys():
        raw_e2rset_dict[o] = set()
    raw_e2rset_dict[s].add(r)
    raw_e2rset_dict[o].add(r+r_num)

    seen_e_set.add(s)
    seen_e_set.add(o)
    seen_r_set.add(r)
    seen_r_set.add(r+r_num)

for key in raw_e2rset_dict.keys():
    raw_e2rset_dict[key] = sorted(raw_e2rset_dict[key])
    raw_r_type_set.add(tuple(raw_e2rset_dict[key]))
    raw_r_type_list.append(tuple(raw_e2rset_dict[key]))
    raw_e_2_type_list.append(key)
    
print(len(seen_e_set))
print(len(seen_r_set))
print(len(raw_r_type_set))# 原始数据集中有多少种不同的关系组合
print(raw_e2rset_dict)
print(raw_r_type_set)
print(seen_e_set)



In [ ]:
# 获取频繁关系子序列
import numpy as np

tmp_set = set()
cover_e_set = set()
cover_r_set = set()
e_list = raw_e_2_type_list #list(raw_e2rset_dict.keys())
sequence_list = raw_r_type_list #list(raw_e2rset_dict.values())
match_num = [0]*len(sequence_list)
tmp_freq_rset = []
raw_data = []
freq_rset = set()

fail_num = 0
patt_len = 3 # 3 for all datsets

def cover_e(patt, matches):
    cover_e = set()
    for i in matches:
        
        if match_num[i[0]] < 5:
            cover_e.add(e_list[i[0]])
            match_num[i[0]] += 1
        else:
            cover_e.add(e_list[i[0]])
            sequence_list[i[0]] = tuple([])
            match_num[i[0]] += 1
        
    tmp_freq_rset.append((len(cover_e), tuple(sorted(patt)), tuple(sorted(cover_e))))

while len(cover_e_set & seen_e_set) != len(seen_e_set):
    tmp_freq_rset = []
    pre_num_cover_e, pre_num_cover_r = len(cover_e_set), len(cover_r_set)

    ps = PrefixSpan(sequence_list)
    ps.topk(1000, closed = True, filter=lambda patt, matches: len(patt) > patt_len and tuple(sorted(patt)) not in freq_rset, callback=cover_e)
    
    tmp_num_cover_e = pre_num_cover_e
    for rset in tmp_freq_rset:
        if rset[1] not in freq_rset:
            cover_r_set = cover_r_set | set(rset[1])
            cover_e_set = cover_e_set | set(rset[2])
            if len(cover_e_set) > tmp_num_cover_e:
                print([len(cover_e_set), len(cover_r_set)])
                tmp_num_cover_e = len(cover_e_set)
                raw_data.append(rset)
                freq_rset.add(rset[1])

    
    if pre_num_cover_e == len(cover_e_set):
        fail_num += 1
    else:
        pre_num_cover_e = len(cover_e_set)
    if fail_num == 3:
        fail_num = 0
        patt_len = patt_len - 1

print(len(freq_rset))

In [ ]:
# 基于集合包含关系合并频繁关系子序列

# 基于类别的集合包含关系合并 O(N*2^k), 优化前为 O(N^2), 2^k << N
sorted_freq_r_list = sorted(raw_data, key = lambda x:len(x[1]), reverse = True)

sparse_type_set = set()
childset_2_fatherset = {}
cover_e_2_freq_r_dict = {}
filtered_freq_r_2_cover_e_dict = {}
for sub_r_set in tqdm(sorted_freq_r_list):
    patt = tuple(sorted(sub_r_set[1]))
    cover_e = sub_r_set[2]
    if 'YAGO' in dataset:
        filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
        continue

    if patt in childset_2_fatherset.keys():
        fathersets = childset_2_fatherset[patt]
        for father_set in fathersets:
            filtered_freq_r_2_cover_e_dict[father_set] = tuple(sorted(set(filtered_freq_r_2_cover_e_dict[father_set]) | set(cover_e)))
    
    else:
        if len(cover_e) >= 5:
            filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
            childsets = powerset(patt, len(patt)+1)
            for child_set in childsets:
                child_set = tuple(sorted(child_set))
                if child_set not in childset_2_fatherset.keys():
                    childset_2_fatherset[child_set] = []
                childset_2_fatherset[child_set].append(patt)
        else:
            childsets = powerset(patt, len(patt))
            flag = 0
            for child_set in childsets:
                child_set = tuple(sorted(child_set))
                if child_set in childset_2_fatherset.keys():
                    flag = 1
                    fathersets = childset_2_fatherset[child_set]
                    for father_set in fathersets:
                        filtered_freq_r_2_cover_e_dict[father_set] = tuple(sorted(set(filtered_freq_r_2_cover_e_dict[father_set]) | set(cover_e)))
            if flag == 0:
                filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(cover_e))
                if len(cover_e) == 1:
                    sparse_type_set.add(patt)

print("Number of sparse types: " + str(len(sparse_type_set)))
print("Sparse types: " + str(sparse_type_set))

for key in filtered_freq_r_2_cover_e_dict.keys():
    value = tuple(sorted(filtered_freq_r_2_cover_e_dict[key]))
    if value not in cover_e_2_freq_r_dict.keys():
        cover_e_2_freq_r_dict[value] = tuple(sorted(key))
    else:
        cover_e_2_freq_r_dict[value] = tuple(sorted(cover_e_2_freq_r_dict[value] | set(key)))

print("Number of types: " + str(len(cover_e_2_freq_r_dict.keys())))

if 'YAGO' not in dataset:
    cover_e_set = set()
    for key in cover_e_2_freq_r_dict.keys():
        cover_e_set = cover_e_set | set(key)
    print(len(cover_e_set & seen_e_set))

    # 基于覆盖实体的集合包含关系合并, 优化前为 O(M^2)
    del_num = 1
    times = 0
    while del_num != 0 and times < 5:
        times += 1
        sorted_cover_e_list = sorted(cover_e_2_freq_r_dict.items(), key = lambda x:len(x[0]), reverse = False)

        del_set = set()
        add_dict = {} #father_cover_e 2 (child_cover_e, r_of_child_cover_e)
        for i in tqdm(range(len(sorted_cover_e_list) - 1)):
            cover_e = sorted_cover_e_list[i][0]
            patt = sorted_cover_e_list[i][1]
            in_i = i+1
            while in_i < len(sorted_cover_e_list):
                in_cover_e = sorted_cover_e_list[in_i][0]
                in_patt = sorted_cover_e_list[in_i][1]
                in_i += 1

                cover_ratio = float(len(set(cover_e) & set(in_cover_e))) / float(min(len(cover_e), len(in_cover_e)))
                if cover_ratio >= 0.8: # 0.8
                    if in_cover_e not in add_dict.keys():
                        add_dict[in_cover_e] = []
                    add_dict[in_cover_e].append([set(cover_e), set(patt)])
                    del_set.add(cover_e)
                else:
                    if len(cover_e) < 5 and len(set(cover_e) - set(in_cover_e)) < len(cover_e):
                        if in_cover_e not in add_dict.keys():
                            add_dict[in_cover_e] = []
                        add_dict[in_cover_e].append([set(cover_e), set(patt)])
                        del_set.add(cover_e)

        del_num = 0
        cover_e_2_freq_r_dict = {}
        for item in sorted_cover_e_list:
            cover_e = item[0]
            patt = item[1]

            if cover_e in add_dict.keys():
                new_cover_e = set(cover_e)
                new_patt = set(patt)
                for in_item in add_dict[cover_e]:
                    new_cover_e = new_cover_e | in_item[0]
                    new_patt = new_patt | in_item[1]
                new_cover_e = tuple(sorted(new_cover_e))
                new_patt = tuple(sorted(new_patt))
                cover_e_2_freq_r_dict[new_cover_e] = new_patt
            else:
                if cover_e in del_set:
                    del_num += 1
                    continue
                else:
                    cover_e_2_freq_r_dict[tuple(cover_e)] = tuple(patt)
    
        print("DEL " + str(del_num) + " types")
        cover_e_set = set()
        for key in cover_e_2_freq_r_dict.keys():
            cover_e_set = cover_e_set | set(key)
        print([len(cover_e_set & seen_e_set), len(cover_e_set), len(seen_e_set)])
                

    # 获取类别—>覆盖实体 字典

    filtered_freq_r_2_cover_e_dict = {}
    for cover_e in cover_e_2_freq_r_dict.keys():
        patt = cover_e_2_freq_r_dict[cover_e]
        if patt not in filtered_freq_r_2_cover_e_dict.keys():
            filtered_freq_r_2_cover_e_dict[patt] = cover_e
        else:
            filtered_freq_r_2_cover_e_dict[patt] = tuple(sorted(set(cover_e) | set(filtered_freq_r_2_cover_e_dict[patt])))

print("Number of filtered types: " + str(len(filtered_freq_r_2_cover_e_dict.keys())))
   

In [ ]:
print(filtered_freq_r_2_cover_e_dict)

cover_e_set = set()
for value in filtered_freq_r_2_cover_e_dict.values():
    cover_e_set = cover_e_set | set(value)
print(len(cover_e_set & seen_e_set))

In [ ]:
# 基于贪心算法求最小集合覆盖
uncover_e_set = seen_e_set
final_r_types = set()
iter_num = 1

while len(uncover_e_set) > 0:
    print("iter" + str(iter_num) + "|" + str(len(final_r_types)) + "|" + str(len(uncover_e_set)))
    iter_num += 1
    max_cover_num = 0
    max_cover_type = None
    max_cover_e = set()
    for r_type in filtered_freq_r_2_cover_e_dict.keys():
        inject_num = len(uncover_e_set & set(filtered_freq_r_2_cover_e_dict[r_type]))
        if inject_num > max_cover_num:
            max_cover_num = inject_num
            max_cover_type = r_type
            max_cover_e = set(filtered_freq_r_2_cover_e_dict[r_type])
    if max_cover_num != 0:
        uncover_e_set = uncover_e_set - max_cover_e
        final_r_types.add(max_cover_type)

print("Final number of types: " + str(len(final_r_types)))

In [ ]:
# 通过约束每个实体的最大类别数消减类别数量
import numpy as np

k = 5 #10 for ICEWS14 ,ICEWS05, YAGO
e_2_type_dict = {}
type_2_e_dict = {}

for key in filtered_freq_r_2_cover_e_dict.keys():
    for e in filtered_freq_r_2_cover_e_dict[key]:
        if e not in e_2_type_dict.keys():
            e_2_type_dict[e] = set()
        e_2_type_dict[e].add(key)

for e in e_2_type_dict.keys():
    old_types = e_2_type_dict[e]
    sorted_old_types = sorted(old_types, key=lambda type: len(filtered_freq_r_2_cover_e_dict[type]), reverse=True)
    new_types = sorted_old_types[:k]
    e_2_type_dict[e] = new_types
    for type in new_types:
        if type not in type_2_e_dict.keys():
            type_2_e_dict[type] = set()
        type_2_e_dict[type].add(e)

r_2_cover_e_num = []
e_2_type_num = []
for key in type_2_e_dict.keys():
    r_2_cover_e_num.append(len(type_2_e_dict[key]))
for key in e_2_type_dict.keys():
    e_2_type_num.append(len(e_2_type_dict[key]))

print(len(type_2_e_dict.keys()))

# 关系的覆盖实体数量统计
print([min(r_2_cover_e_num), np.mean(r_2_cover_e_num), max(r_2_cover_e_num)])

# 实体的关系数量统计
print([min(e_2_type_num), np.mean(e_2_type_num), max(e_2_type_num)])


In [ ]:
print(type_2_e_dict)

In [ ]:
# 存储类别文件
import numpy as np

type_2_id = dict()
id = 0

f = open(root_path+dataset+'types.txt','w', encoding='utf-8')
for type in type_2_e_dict.keys():
    type_2_id[type] = id
    f.writelines(str(type) + '	' + str(id) + '\n')
    id += 1
f.close()

e_2_type_id = dict()
f = open(root_path+dataset+'e2types.txt','w', encoding='utf-8')
for e in e_2_type_dict.keys():
    types = e_2_type_dict[e]
    ids = [type_2_id[type] for type in types]
    e_2_type_id[e] = ids
    f.writelines(str(e) + '	' + str(ids) + '\n')
f.close()

print(len(type_2_id.items()))
